# Env

In [1]:
import argparse
import random

import numpy as np
import torch

In [2]:
# 환경 설정
args = {
    # random seed value
    "seed": 1234,
    # CPU 또는 GPU 사용여부 결정
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}
args = argparse.Namespace(**args)

print(args)

Namespace(device=device(type='cpu'), seed=1234)


In [3]:
# random seed 설정
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

# Data & Vocabulary

In [4]:
# 입력 문장
raw_inputs = [
    "나는 학생 입니다",
    "나는 좋은 선생님 입니다",
    "당신은 매우 좋은 선생님 입니다"
]

# 정답: 학생(1), 기타(0)
raw_labels = [1, 0, 0]

In [5]:
# 문장을 띄어쓰기단위로 분할
words = []
for s in raw_inputs:
    words.extend(s.split())

# 중복단어 제거
words = list(dict.fromkeys(words))

# 각 단어별 일련번호
word_to_id = {"[PAD]": 0, "[UNK]": 1}
for w in words:
    word_to_id[w] = len(word_to_id)

# 각 번호별 단어
id_to_word = {i: w for w, i in word_to_id.items()}

print(id_to_word)

{0: '[PAD]', 1: '[UNK]', 2: '나는', 3: '학생', 4: '입니다', 5: '좋은', 6: '선생님', 7: '당신은', 8: '매우'}


# One-hot Encoding

In [9]:
# 일련번호 데이터
train_inputs = []
for s in raw_inputs:
    row = [word_to_id[w] for w in s.split()]
    row += [0] * (5 - len(row))
    train_inputs.append(row)
train_inputs = np.array(train_inputs)

print(train_inputs)

[[2 3 4 0 0]
 [2 5 6 4 0]
 [7 8 5 6 4]]


In [7]:
# one-hot metrix 생성
onehot_metrix = np.eye(len(word_to_id))

print(onehot_metrix)

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [11]:
# 일련번호를 one-hot으로 변경
train_onehots = onehot_metrix[train_inputs]

print(train_onehots)

[[[0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]]]


In [12]:
# one-hot을 일련번호로 변경
print(np.argmax(train_onehots, axis=-1))

[[2 3 4 0 0]
 [2 5 6 4 0]
 [7 8 5 6 4]]


# Embedding

In [15]:
# torch tensor로 변경
t_train_inputs = torch.tensor(train_inputs)
t_train_onehots = torch.tensor(train_onehots).to(torch.float32)

In [16]:
# Embedding 선언
embed = torch.nn.Embedding(len(word_to_id), 4)

In [17]:
# 일련번호를 단어 벡터로 변경
hidden1 = embed(t_train_inputs)

print(hidden1)

tensor([[[-0.2153,  0.8840, -0.7584, -0.3689],
         [-0.3424, -1.4020,  0.3206, -1.0219],
         [ 0.7988, -0.0923, -0.7049, -1.6024],
         [-0.1117, -0.4966,  0.1631, -0.8817],
         [-0.1117, -0.4966,  0.1631, -0.8817]],

        [[-0.2153,  0.8840, -0.7584, -0.3689],
         [-0.5675, -0.2772, -2.1834,  0.3668],
         [ 0.7667,  0.0190,  0.0220,  1.1532],
         [ 0.7988, -0.0923, -0.7049, -1.6024],
         [-0.1117, -0.4966,  0.1631, -0.8817]],

        [[ 1.8409, -1.0174,  1.2192,  0.1601],
         [-0.6857, -0.0496, -1.2485, -0.8509],
         [-0.5675, -0.2772, -2.1834,  0.3668],
         [ 0.7667,  0.0190,  0.0220,  1.1532],
         [ 0.7988, -0.0923, -0.7049, -1.6024]]], grad_fn=<EmbeddingBackward>)


In [18]:
# embedding wieght
print(embed.weight)

Parameter containing:
tensor([[-0.1117, -0.4966,  0.1631, -0.8817],
        [ 0.0539,  0.6684, -0.0597, -0.4675],
        [-0.2153,  0.8840, -0.7584, -0.3689],
        [-0.3424, -1.4020,  0.3206, -1.0219],
        [ 0.7988, -0.0923, -0.7049, -1.6024],
        [-0.5675, -0.2772, -2.1834,  0.3668],
        [ 0.7667,  0.0190,  0.0220,  1.1532],
        [ 1.8409, -1.0174,  1.2192,  0.1601],
        [-0.6857, -0.0496, -1.2485, -0.8509]], requires_grad=True)


In [19]:
# one_hots와 wegith와 matmul
hidden2 = torch.matmul(t_train_onehots, embed.weight)

print(hidden2)

tensor([[[-0.2153,  0.8840, -0.7584, -0.3689],
         [-0.3424, -1.4020,  0.3206, -1.0219],
         [ 0.7988, -0.0923, -0.7049, -1.6024],
         [-0.1117, -0.4966,  0.1631, -0.8817],
         [-0.1117, -0.4966,  0.1631, -0.8817]],

        [[-0.2153,  0.8840, -0.7584, -0.3689],
         [-0.5675, -0.2772, -2.1834,  0.3668],
         [ 0.7667,  0.0190,  0.0220,  1.1532],
         [ 0.7988, -0.0923, -0.7049, -1.6024],
         [-0.1117, -0.4966,  0.1631, -0.8817]],

        [[ 1.8409, -1.0174,  1.2192,  0.1601],
         [-0.6857, -0.0496, -1.2485, -0.8509],
         [-0.5675, -0.2772, -2.1834,  0.3668],
         [ 0.7667,  0.0190,  0.0220,  1.1532],
         [ 0.7988, -0.0923, -0.7049, -1.6024]]], grad_fn=<UnsafeViewBackward>)
